In the second week, we have to go with a input system and face an online on naive bayes classifier.
In the first line there are three numbers : Number of features, Number of classes and Number of samples.

In [1]:
import numpy as np
import math

Function for loading data from file

In [2]:

features=0
classes=0
samples=0

def data_loader(filename,isTrainData):
    # open data file
    file = open("instructions and data/during coding/"+filename,"r")


    # initialize
    i=0
    global features
    global classes
    global samples


    listx = []
    listy = []

    for line in file:
        # for the first line
        if(i==0 and isTrainData==1):
            fields = line.split()

            features = int(fields[0])
            classes = int(fields[1])
            samples = int(fields[2])
        # for the rest of the line
        else:
            fields = line.split()
            templist = []

            for j in range(features):
                #print(fields[j])
                templist.append(float(fields[j]))

            listx.append(templist)
            listy.append(int(fields[features]))

        i = i+1

    #print(str(features)+" "+str(classes)+" "+str(samples))

    # convert into numpy array
    x = np.array(listx)
    y = np.array(listy)

    #print(x.shape, y.shape)
    
    return x,y


Load train data

In [3]:
train_x,train_y =  data_loader("Train.txt",1)
print(train_x.shape,train_y.shape)

(100, 2) (100,)


Load test data

In [4]:
test_x,test_y =  data_loader("Test.txt",0)
print(test_x.shape,test_y.shape)

(100, 2) (100,)


In [5]:
print("Features: "+str(features)+"\nClasses: "+str(classes)+"\nSamples: "+str(samples))

Features: 2
Classes: 2
Samples: 100


Calculating prior probability

In [6]:
pre_prob = np.zeros((1, classes))

for i in range(classes):
    total = (train_y==(i+1)).sum()
    pre_prob[0][i] = total/(samples)
print(pre_prob)

[[0.5 0.5]]


calculate mean and deviation

In [19]:
mean = np.zeros((classes,features))
deviation = np.zeros((classes,features))
covariance = np.zeros((classes,features,features))

for i in range(features):
    for j in range(classes):
        value = 0.0
        for k in range(samples):
            if(train_y[k]==(j+1)):
                value =  value + train_x[k][i]
        mean[j][i]=value/(train_y==(j+1)).sum()
print(mean)
for i in range(features):
    for j in range(classes):
        value = 0.0
        for k in range(samples):
            if(train_y[k]==(j+1)):
                value = value+ (train_x[k][i] - mean[j][i])*(train_x[k][i] - mean[j][i])
        deviation[j][i]=value/(train_y==(j+1)).sum()
print(deviation)

for x in range(classes):
    for i in range(features):
        for j in range(features):
            value = 0.0
            for k in range(samples):
                if(train_y[k]==(x+1)):
                    value = value+ (train_x[k][i] - mean[x][i])*(train_x[k][j] - mean[x][j])
            covariance[x][i][j]=value/(features)
print(covariance)



[[ 2.01143   4.057054]
 [ 7.803882 10.073138]]
[[0.25451926 0.25819941]
 [0.17793744 0.19076063]]
[[[ 6.3629815  -0.82428432]
  [-0.82428432  6.45498514]]

 [[ 4.44843596  0.37718217]
  [ 0.37718217  4.76901583]]]


Caluculate gaussian distribution

In [21]:
def gauss(value,class_no):
    global mean
    global covariance
    value_min = np.zeros((features,1))
    for i in range(features):
        value_min[i][0] = (value[i]-mean[i][class_no])
    
    value_min_trans = value_min.transpose()
    
    covar = np.dot(value_min,value_min_trans)
    #print(covariance.shape)
    
    det = np.linalg.det(covariance[class_no])
    #print(covariance)
    inverse = np.linalg.inv(covar)
    p = np.dot(inverse,value_min)
    
    expo = np.dot(value_min_trans,p)
    
    expo_val = expo[0][0]*-.5
    
    expo_val = math.exp(expo_val)
    #print(expo_val)
    co_eff = expo_val/(((2*3.14159)**(features/2.0))* (abs(det)**0.5))
    print(co_eff)
    return co_eff

Calculating for test data

In [22]:
predict = np.zeros((samples,1))
for i in range(samples):
    
    prob_list = []
    for j in range(classes):
        value =  gauss(test_x[i],j)
        prob_list.append(value*pre_prob[0][j])
    pos = prob_list.index(max(prob_list))
    predict[i] = pos+1
#print(predict)

0.006211216604706723
0.036592438097502754
0.025041752638635853
0.01269822836230842
0.025041752638635853
0.034670774883363356
0.013935848338515574
2.7117017468630978e-06
0.07230002951094933
2.2577992929197598e-08
0.025041752638635853
0.034670774883363356
0.005289927482362774
0.023903824921826793
0.014229564216287571
0.014835416957776555
0.014502470835213622
0.01573844168671568
0.01400265265939677
0.01879392369166502
0.020135894880390984
0.009039194890954472
0.004973150473349164
0.020743119673740148
0.42473461446858224
0.034670774883363356
0.01292049147301759
0.017657210638357127
0.013365603947888435
0.023164906548737884
0.0026695142775389133
0.034670774883363356
0.2134717568152068
0.034670774883363356
0.007534590900238302
0.02293727885806019
0.00671325907565678
0.06816018588496209
0.0103512189667137
0.0242923985540361
0.023790272706888334
0.007914162371142733
0.017162118417406728
0.026690037266909633
0.025041752638635853
0.036550285036543395
0.010870245514821951
0.019007499459851772
0.0

Calculate accuracy

In [23]:
correct = 0
for i in range(samples):
    if test_y[i] == int(predict[i][0]):
        correct = correct+1
    else:
        feature_str=""
        for j in range(features):
            feature_str = feature_str +str(test_x[i][j])
        print(str(i+1)+" "+feature_str +" "+str(test_y[i])+" "+str(int(predict[i][0])))
print("Accuracy : " +str((correct*100)/samples))

1 2.46613.9251 1 2
3 1.25074.1563 1 2
6 2.04173.2886 1 2
7 2.78884.1234 1 2
8 1.83463.2821 1 2
9 2.39764.0743 1 2
10 1.60763.1535 1 2
12 2.33334.5709 1 2
14 1.34974.6918 1 2
15 1.69753.621 1 2
16 1.25574.2213 1 2
18 1.86133.463 1 2
19 1.35324.1009 1 2
20 1.55584.3814 1 2
22 1.96423.5235 1 2
23 0.79274.3891 1 2
24 1.65283.9968 1 2
26 2.16484.6821 1 2
27 2.29934.241 1 2
30 0.68253.9166 1 2
31 2.0143.5919 1 2
34 1.83623.5314 1 2
36 2.294.841 1 2
37 2.11994.2968 1 2
38 1.82464.3951 1 2
41 1.44594.4355 1 2
45 2.25013.6573 1 2
46 1.74143.8658 1 2
48 1.62334.1243 1 2
52 1.74461.7209 2 1
53 8.124610.3361 2 1
54 8.184610.0693 2 1
56 7.98149.6239 2 1
60 8.242610.4031 2 1
62 7.9579.8133 2 1
66 7.80889.2135 2 1
67 7.523311.0078 2 1
68 8.11689.964 2 1
71 7.749210.0866 2 1
72 8.361410.4616 2 1
73 8.01979.9107 2 1
74 8.77069.7391 2 1
75 7.149510.716 2 1
76 7.48319.5649 2 1
77 7.618110.4038 2 1
78 9.08829.7447 2 1
79 8.215810.3718 2 1
80 7.778110.4239 2 1
81 8.0159.585 2 1
88 7.94329.8625 2 1
90 7.600